## Project 1: Robot Path Planning 

In [295]:
# Import libaries
import math
import heapq
import altair as alt
import pandas as pd

In [296]:
# Read in input file store start coordinate, end coordinate, and the grid matrix
def read_input_file(file_path):
    with open(file_path, 'r') as file:
        first_line = file.readline().strip()
        start_x, start_y, goal_x, goal_y = map(int, first_line.split())
        grid_matrix = []
        for y in range(29, -1, -1):  
            line = file.readline().strip()
            if line: 
                cell_values = list(map(int, line.split()))
                grid_matrix.append([[x, y, cell_values[x]] for x in range(len(cell_values))])

    return (start_x, start_y), (goal_x, goal_y), grid_matrix

start_coord, goal_coord, grid_matrix = read_input_file("input1.txt")
print(start_coord)
print(goal_coord)

(3, 2)
(10, 6)
[[0, 29, 0], [1, 29, 0], [2, 29, 0], [3, 29, 0], [4, 29, 0], [5, 29, 0], [6, 29, 0], [7, 29, 0], [8, 29, 0], [9, 29, 0], [10, 29, 0], [11, 29, 0], [12, 29, 0], [13, 29, 0], [14, 29, 0], [15, 29, 0], [16, 29, 0], [17, 29, 0], [18, 29, 0], [19, 29, 0], [20, 29, 0], [21, 29, 0], [22, 29, 0], [23, 29, 0], [24, 29, 0], [25, 29, 0], [26, 29, 0], [27, 29, 0], [28, 29, 0], [29, 29, 0], [30, 29, 0], [31, 29, 0], [32, 29, 0], [33, 29, 0], [34, 29, 0], [35, 29, 0], [36, 29, 0], [37, 29, 0], [38, 29, 0], [39, 29, 0], [40, 29, 0], [41, 29, 0], [42, 29, 0], [43, 29, 0], [44, 29, 0], [45, 29, 0], [46, 29, 0], [47, 29, 0], [48, 29, 0], [49, 29, 0]]
[[0, 28, 0], [1, 28, 0], [2, 28, 0], [3, 28, 0], [4, 28, 0], [5, 28, 0], [6, 28, 0], [7, 28, 0], [8, 28, 0], [9, 28, 0], [10, 28, 0], [11, 28, 0], [12, 28, 0], [13, 28, 0], [14, 28, 0], [15, 28, 0], [16, 28, 0], [17, 28, 0], [18, 28, 0], [19, 28, 0], [20, 28, 0], [21, 28, 0], [22, 28, 0], [23, 28, 0], [24, 28, 0], [25, 28, 0], [26, 28, 0], [2

In [297]:
import pandas as pd
import altair as alt

# Use the grid_matrix from the read_input_file function
flat_grid = [cell for row in grid_matrix for cell in row]
df = pd.DataFrame(flat_grid, columns=['x', 'y', 'cell_value'])

# Define color scale for heatmap
color_scale = alt.Scale(domain=[0, 1, 2, 5], range=['white', 'black', 'red', 'green'])

# Create the heatmap
heatmap = alt.Chart(df).mark_rect().encode(
    x=alt.X('x:O', axis=alt.Axis(title='x', values=list(range(0, 51, 1)))),
    y=alt.Y('y:O', axis=alt.Axis(title='y', values=list(range(0, 31, 1))), sort='descending'),
    color=alt.Color('cell_value:N', scale=color_scale)
).properties(
    width=800,
    height=400
)

# Adding horizontal grid lines
horizontal_grid = alt.Chart(pd.DataFrame({'y': [i + 0.5 for i in range(31)]})).mark_rule(color='gray', strokeWidth=0.5).encode(
    y=alt.Y('y:O', sort='descending')  # Ensure grid lines respect the y-axis ordering
)

# Adding vertical grid lines
vertical_grid = alt.Chart(pd.DataFrame({'x': [i + 0.5 for i in range(51)]})).mark_rule(color='gray', strokeWidth=0.5).encode(
    x='x:O'
)

# Combine the heatmap with the grid lines
heatmap = heatmap + horizontal_grid + vertical_grid

heatmap

alt.LayerChart(...)

In [298]:
class Node:
    def __init__(self, coord, parent=None, action=None, g=0, h=0):
        self.coord = coord
        self.parent = parent
        self.action = action
        self.g = g  
        self.h = h  
    
    def f(self):
        return self.g + self.h
    
    def __lt__(self, other):
        return self.f() < other.f()

actions = [(1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1)]

def is_valid_move(x, y, grid_dict):
    print(grid_dict.get((x, y), None))
    return 0 <= x < 50 and 0 <= y < 30 and grid_dict.get((x, y), None) == 0

def euclidean_distance(coord1, coord2):
    x1, y1 = coord1
    x2, y2 = coord2
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

def a_star_search(start_coord, goal_coord, grid_dict):
    frontier = []
    visited = set()
    start_node = Node(start_coord, None, None, 0, euclidean_distance(start_coord, goal_coord))
    heapq.heappush(frontier, start_node)
    
    while frontier:
        current_node = heapq.heappop(frontier)
        visited.add(current_node.coord)
        # print("current node:" + str(current_node.coord))
        
        if current_node.coord == goal_coord:
            return current_node
        
        for action in actions:
            new_x = current_node.coord[0] + action[0]
            new_y = current_node.coord[1] + action[1]
            # print("new x, y: " + str((new_x, new_y)))
            if is_valid_move(new_x, new_y, grid_dict) and (new_x, new_y) not in visited:
                new_g = current_node.g + (1 if action[0] == 0 or action[1] == 0 else math.sqrt(2))
                new_h = euclidean_distance((new_x, new_y), goal_coord)
                new_node = Node((new_x, new_y), current_node, action, new_g, new_h)
                # print("new_node.coord: " + str(new_node.coord))
                heapq.heappush(frontier, new_node)
    
    return None

def extract_path(goal_node):
    path = []
    f_values = []
    current_node = goal_node
    while current_node is not None:
        path.append(current_node.action)
        f_values.append(current_node.f())
        current_node = current_node.parent
    path.reverse()
    f_values.reverse()
    return path, f_values

def write_output_file(goal_node, grid_dict, output_file):
    path, f_values = extract_path(goal_node)
    depth_level = len(path) - 1
    total_nodes = len(path) + 1
    
    with open(output_file, 'w') as file:
        file.write(str(depth_level) + '\n')
        file.write(str(total_nodes) + '\n')
        file.write(' '.join(str(action) for action in path) + '\n')
        file.write(' '.join(str(f) for f in f_values) + '\n')
        for y in range(29, -1, -1):  
            row_values = [str(grid_dict.get((x, y), 0)) for x in range(50)]
            file.write(' '.join(row_values) + '\n')

In [299]:
grid_dict = {(x, y): value for row in grid_matrix for x, y, value in row}
goal_node = a_star_search(start_coord, goal_coord, grid_dict)
if goal_node:
    write_output_file(goal_node, grid_dict, "output.txt")
    print("Output written to output.txt")
else:
    print("No path found")

current node:(3, 2)
new x, y: (4, 2)
[4, 27, 0]
new x, y: (4, 3)
[4, 26, 0]
new x, y: (3, 3)
[3, 26, 0]
new x, y: (2, 3)
[2, 26, 0]
new x, y: (2, 2)
[2, 27, 0]
new x, y: (2, 1)
[2, 28, 0]
new x, y: (3, 1)
[3, 28, 0]
new x, y: (4, 1)
[4, 28, 0]
No path found
